In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [74]:
out='../output/'
inb='../input/flows/'
pref='fpm'
inp = inb + pref + '/'

In [75]:
dat=pd.DataFrame(columns=['date','flow','from','hour','to'])
onlyfiles = [f for f in listdir(inp ) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat['flow']=dat['flow'].astype(int)
dat

,date,flow,from,hour,to
0,2016-04-01,10,BOT,0,ZIMA
1,2016-04-01,10,MOZN_EDM,0,ZIMA
2,2016-04-01,10,MOZN_HCB,0,MOZN_EDM
3,2016-04-01,10,MOZS,0,RSAN
4,2016-04-01,10,RSAN,0,BOT
...,...,...,...,...,...
2233,2021-10-31,20,MOZS,23,RSAN
2234,2021-10-31,20,RSAN,23,BOT
2235,2021-10-31,20,ZAMZ,23,ZAMC
2236,2021-10-31,20,ZIM,23,ZAMZ


In [76]:
dam = pd.read_csv(out + pref + '_hourly.csv')
dam=dam.set_index([pd.DatetimeIndex(dam['date']),'hour'])
dam=dam.drop(['date'],1)
dam

price  volume  year  month  day  day_of_week    value  \
date       hour                                                          
2017-04-01 4     39.06     9.4  2017      4    1            5  367.164   
           5     39.06     9.4  2017      4    1            5  367.164   
           6     39.06     9.4  2017      4    1            5  367.164   
           7     67.32     0.0  2017      4    1            5    0.000   
           8     67.32     0.0  2017      4    1            5    0.000   
...                ...     ...   ...    ...  ...          ...      ...   
2023-01-31 19    90.00     0.0  2023      1   31            1    0.000   
           20    85.00     0.0  2023      1   31            1    0.000   
           21    85.00     0.0  2023      1   31            1    0.000   
           22    62.00     0.0  2023      1   31            1    0.000   
           23    62.00     0.0  2023      1   31            1    0.000   

                      cat  
date       hour            
2017-04-01 4          off  
           5          off  
           6          off  
           7     standard  
           8     standard  
...                   ...  
2023-01-31 19    standard  
           20    standard  
           21    standard  
           22         off  
           23         off  

[45980 rows x 8 columns]

In [77]:
flow=dat
#flow['date']=pd.to_datetime(flow['flowdate'],format="%d/%m/%Y") #"%Y-%m-%d")#"%d/%m/%Y")
#flow=flow.drop(['flowdate','flowid','market'],axis=1)

flow=flow.set_index([pd.DatetimeIndex(flow['date'])])
flow['year']=flow.index.year
flow['month']=flow.index.month
flow['day']=flow.index.day
flow['day_of_week']=flow.index.dayofweek

flow=flow.set_index([pd.DatetimeIndex(flow['date']),'hour'])
flow['price']=dam['price']
flow['value']=flow['price']*flow['flow']
flow=flow.drop(['date'],axis=1)
flow=flow.dropna(0)
flow=flow.reset_index()

flow

,date,hour,flow,from,to,year,month,day,day_of_week,price,value
0,2017-04-10,4,9,BOT,ZIMA,2017,4,10,0,39.06,351.54
1,2017-04-10,4,9,RSAN,BOT,2017,4,10,0,39.06,351.54
2,2017-04-10,4,9,ZAMZ,ZAMC,2017,4,10,0,39.06,351.54
3,2017-04-10,4,9,ZIM,ZAMZ,2017,4,10,0,39.06,351.54
4,2017-04-10,4,9,ZIMA,ZIM,2017,4,10,0,39.06,351.54
...,...,...,...,...,...,...,...,...,...,...,...
68478,2021-10-31,23,20,MOZS,RSAN,2021,10,31,6,40.05,801.00
68479,2021-10-31,23,20,RSAN,BOT,2021,10,31,6,40.05,801.00
68480,2021-10-31,23,20,ZAMZ,ZAMC,2021,10,31,6,40.05,801.00
68481,2021-10-31,23,20,ZIM,ZAMZ,2021,10,31,6,40.05,801.00


In [78]:
f_to=flow[['date','hour','flow','value','price','to','year','month','day','day_of_week']].rename(columns={'to': 'node'})
f_from=flow[['date','hour','flow','value','price','from','year','month','day','day_of_week']].rename(columns={'from': 'node'})
f_from['flow']=-f_from['flow']
f_from['value']=-f_from['value']

flows=pd.concat([f_to, f_from])

flows

,date,hour,flow,value,price,node,year,month,day,day_of_week
0,2017-04-10,4,9,351.54,39.06,ZIMA,2017,4,10,0
1,2017-04-10,4,9,351.54,39.06,BOT,2017,4,10,0
2,2017-04-10,4,9,351.54,39.06,ZAMC,2017,4,10,0
3,2017-04-10,4,9,351.54,39.06,ZAMZ,2017,4,10,0
4,2017-04-10,4,9,351.54,39.06,ZIM,2017,4,10,0
...,...,...,...,...,...,...,...,...,...,...
68478,2021-10-31,23,-20,-801.00,40.05,MOZS,2021,10,31,6
68479,2021-10-31,23,-20,-801.00,40.05,RSAN,2021,10,31,6
68480,2021-10-31,23,-20,-801.00,40.05,ZAMZ,2021,10,31,6
68481,2021-10-31,23,-20,-801.00,40.05,ZIM,2021,10,31,6


In [79]:
flows.loc[flows['node']=='ZIMA','node']='ZIM'
flows.loc[flows['node']=='MOZN_EDM','node']='MOZ'
flows.loc[flows['node']=='MOZN_HCB','node']='HCB'
flows.loc[flows['node']=='MOZS','node']='MOZ'
flows.loc[flows['node']=='RSAS','node']='RSA'
flows.loc[flows['node']=='RSAN','node']='RSA'

In [80]:
flows=flows.reset_index().groupby(['date','hour','node','year','month','day','day_of_week']).sum().reset_index()
flows=flows.loc[flows['flow']!=0]
flows=flows.set_index(['date','hour'])
flows['cat']=dam['cat']
flows.reset_index()
flows

node  year  month  day  day_of_week  index  flow   value  \
date       hour                                                             
2017-04-10 4      RSA  2017      4   10            0      1    -9 -351.54   
           4     ZAMC  2017      4   10            0      2     9  351.54   
           5      RSA  2017      4   10            0      6    -9 -351.54   
           5     ZAMC  2017      4   10            0      7     9  351.54   
           22     RSA  2017      4   10            0     11    -9 -351.54   
...               ...   ...    ...  ...          ...    ...   ...     ...   
2021-10-31 21    ZAMC  2021     10   31            6  68468    20  801.00   
           22     MOZ  2021     10   31            6  68472   -20 -801.00   
           22    ZAMC  2021     10   31            6  68474    20  801.00   
           23     MOZ  2021     10   31            6  68478   -20 -801.00   
           23    ZAMC  2021     10   31            6  68480    20  801.00   

                 price  cat  
date       hour              
2017-04-10 4     39.06  off  
           4     39.06  off  
           5     39.06  off  
           5     39.06  off  
           22    39.06  off  
...                ...  ...  
2021-10-31 21    40.05  off  
           22    40.05  off  
           22    40.05  off  
           23    40.05  off  
           23    40.05  off  

[25214 rows x 10 columns]

In [81]:
flowsb=flows.copy()
flowsb.loc[flowsb['flow']<0,'side']='supply'
flowsb.loc[flowsb['flow']>0,'side']='consume'


flow_monthly_cat=flowsb.groupby(['year','month', 'node','cat','side']).sum()
flow_monthly_cat.rename(columns={'flow': 'flow_sum'}, inplace=True)
flow_monthly_cat['hour_count']=flowsb.groupby(['year','month', 'node','cat','side']).count()['flow']
flow_monthly_cat['price_weight_mean'] = flow_monthly_cat['value'] / flow_monthly_cat['flow_sum']
flow_monthly_cat['price_mean']=flowsb.groupby(['year','month', 'node','cat','side']).mean()['price']
flow_monthly_cat=flow_monthly_cat.drop(['day','day_of_week','index','price'],axis=1)
flow_monthly_cat=flow_monthly_cat.loc[flow_monthly_cat['flow_sum']!=0]
flow_monthly_cat['power']=flow_monthly_cat['flow_sum']/flow_monthly_cat['hour_count']
flow_monthly_cat

flow_sum      value  hour_count  \
year month node cat      side                                       
2017 4     RSA  off      supply      -2079  -81205.74         231   
           ZAMC off      consume      2079   81205.74         231   
     5     RSA  off      supply      -3160 -117141.20         316   
                standard supply       -160   -5931.20          16   
           ZAMC off      consume      3160  117141.20         316   
...                                    ...        ...         ...   
2021 8     ZAMC off      consume      6640  398001.60         332   
     9     MOZ  off      supply      -6240 -268008.00         312   
           ZAMC off      consume      6240  268008.00         312   
     10    MOZ  off      supply      -6820 -273141.00         341   
           ZAMC off      consume      6820  273141.00         341   

                                  price_weight_mean  price_mean  power  
year month node cat      side                                           
2017 4     RSA  off      supply               39.06       39.06   -9.0  
           ZAMC off      consume              39.06       39.06    9.0  
     5     RSA  off      supply               37.07       37.07  -10.0  
                standard supply               37.07       37.07  -10.0  
           ZAMC off      consume              37.07       37.07   10.0  
...                                             ...         ...    ...  
2021 8     ZAMC off      consume              59.94       59.94   20.0  
     9     MOZ  off      supply               42.95       42.95  -20.0  
           ZAMC off      consume              42.95       42.95   20.0  
     10    MOZ  off      supply               40.05       40.05  -20.0  
           ZAMC off      consume              40.05       40.05   20.0  

[109 rows x 6 columns]

In [82]:
flow_monthly_cat.to_csv(out + pref + '_flow_monthly_cat.csv')